### Calculating Similarity

The inspiration for this approach originates from a reading of this article:

###### BERT For Measuring Text Similarity
[High-performance semantic similarity with BERT](https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1)

Our approach will be to read all the summaries from the articles data base, use them to encode and create embeddings and compare the candidate articles to them, then observe scores.

In [2]:
import sys
import os
# LIBRARY_PATH = '/home/deleidos/Notebooks/Newspaper/SemanticEstimation'
LIBRARY_PATH = "/home/deleidos/Notebooks/lib"
module_path = os.path.abspath(os.path.join(LIBRARY_PATH))
if module_path not in sys.path:
    sys.path.append(module_path)
print (f"sys.path : {sys.path}") 


sys.path : ['/home/deleidos/Notebooks/Newspaper/SemanticEstimation', '/opt/conda/lib/python38.zip', '/opt/conda/lib/python3.8', '/opt/conda/lib/python3.8/lib-dynload', '', '/opt/conda/lib/python3.8/site-packages', '/opt/conda/lib/python3.8/site-packages/IPython/extensions', '/home/deleidos/.ipython', '/home/deleidos/Notebooks/Newspaper', '/home/deleidos/Notebooks/lib']


In [3]:
import torch

import os, csv
from csv import writer, DictWriter
from os import listdir
import os.path as osp
from os.path import isfile, isdir, join

import newspaper
from newspaper import Article 
from keybert import KeyBERT

import json 
import boto3 
import logging 
import uuid 
import time
import datetime
from datetime import datetime as DT
from pathlib import Path 
import os, sys 
import glob
import nltk
from bertopic import BERTopic
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import pipeline
import pandas as pd
import numpy as np
import contractions

# nltk.download('punkt')
# setup widget import
from ipywidgets import widgets,Layout
from IPython.display import display

module_path = str(Path.cwd().parents[0] / "")
if module_path not in sys.path:
    sys.path.append(module_path) 

#LOCALS
RESOURCE = 'dynamodb'
TABLE = "Articles-tfoo6wfq2zbxfdwl3z47nsgtrq-dev"
REGION = "us-east-1"

CSVPath = "/home/deleidos/Notebooks/CSVData"
JSONPath = "/home/deleidos/Notebooks/JSONData"
MODELPATH= "/home/deleidos/Notebooks/Models"
TRANSFORMERMODEL = 'all-MiniLM-L6-v2'
from dynamoDButils import *
from APIutils import *
from seedterms import *
from utils import *

import random
from timeit import default_timer as timer
import time
import datetime
from datetime import datetime as DT

def complete(state="complete"):
    print(f"\nCell {state} @ {(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))}")
complete(state='imports done')

[nltk_data] Downloading package punkt to /home/deleidos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Cell imports done @ 2023-02-03 19:32:38


In [4]:
%%time
# Anna , if you want try different transformers?
model = SentenceTransformer(TRANSFORMERMODEL)
complete(state='model initialised')


Cell model initialised @ 2023-02-03 19:32:40
CPU times: user 467 ms, sys: 137 ms, total: 603 ms
Wall time: 601 ms


Next choose the corpus of news articles 

In [5]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsn2WD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsn2WD.observe(on_change)
display(jsn2WD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [6]:
# iWD.value
jsnf = jsn2WD.value
print (f"{jsnf=} ")

# jsnfile = osp.join (JSONPath,jsnf)
# xposedf = pd.read_json(jsnfile)
# xposedf = xposedf.transpose() # pivot the matrix
# xposedf

jsnf='semanticSimArticlesME.json' 


#### Read Corpus  articles and create sentence embeddings
Here, we begin the process of looking at a collection of articles and rating them for inclusion vice a model constructed from the existing corpus of articles

In [8]:
jsnfile = osp.join (JSONPath,jsnf)
cSummaries = pd.read_json(jsnfile)
cSummaries = cSummaries.transpose() # pivot the matrix
cSummaries

,url,title,summary
0,https://www.cnn.com/2023/01/16/politics/overcl...,Analysis: There's a larger 'classified' proble...,A version of this story appeared in CNN’s What...
1,https://www.techrepublic.com/article/best-digi...,Best digital collaboration tools for your busi...,These 10 real-time digital collaboration apps ...
2,https://www.businessinsider.com/i-asked-chatgp...,I asked ChatGPT to do my work and write an Ins...,"I asked ChatGPT, a new AI chatbot from OpenAI,..."
3,https://www.engadget.com/netflixs-dog-and-boy-...,Netflix's 'Dog and Boy' anime causes outrage f...,"In 2016, Studio Ghibli co-founder and director..."
4,https://www.macworld.com/article/1484137/apple...,Apple’s next battle in its ‘silent war’ agains...,While we’re all waiting for Apple to unveil it...
...,...,...,...
3539,https://www.cnn.com/2023/01/10/asia/china-fune...,China: Satellite images capture crowding at cr...,CNN —Satellite images taken over a number of C...
3540,https://www.techrepublic.com/article/nrf-2023-...,NRF 2023: How ambient computing could change r...,Wiliot’s cloud-connected tags attempt to solve...
3541,https://www.macworld.com/article/1476747/homep...,"Defying logic, Apple announces 2nd-gen HomePod...",A day after unveiling updated Macs with M2 pro...
3542,https://www.digitaltrends.com/dtdeals/lenovo-l...,Lenovo Legion gaming PCs and laptops are on sa...,"As we’re moving through January, we keep seein..."


In [9]:
cSentences = []
cSentences = cSummaries.summary.to_list()

complete(state="models")
cSentences[0]


Cell models @ 2023-02-03 19:38:17


'A version of this story appeared in CNN’s What Matters newsletter. To get it in your inbox, sign up for free here.CNN —This is getting absurd.Americans have had the opportunity to choose from between three people to be president since 2016, and all three have been found to have classified documents they should not have.Clearly, there’s a larger problem.Here’s what we don’t know:We, the public, don’t know what documents President Joe Biden had locked up in his garage or in the manila folder in his Penn Biden Center office.We don’t know specifics about what documents the FBI photographed on the floor at former President Donald Trump’s Mar a Lago resort.We don’t even know for sure what documents were on Hillary Clinton’s email server, although we do know that dozens of those documents were retroactively classified years after the fact, long after the existence of her private email server became a political albatross.Here’s what we do know:Classified documents, and their appearance in ema

#### Candidates for inclusion
Next read in the candidate collection and use it to create an embedding

In [10]:
jsonfiles = [f for f in listdir(JSONPath) if isfile(join(JSONPath, f))]
jsonfiles.sort()
jsonfiles = [x for x in jsonfiles if (os.path.splitext(x)[1][1:]).lower() == "json"]
jsnWD = widgets.Select(
    options=jsonfiles,
    value=jsonfiles[0],
    description='Choose ID:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        layout=Layout(width='50%', height='256px')
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
#         print ("changed to %s" % change['new'])
        selection = change['new']

jsnWD.observe(on_change)
display(jsnWD)

Select(description='Choose ID:', layout=Layout(height='256px', width='50%'), options=('2000CWE.json', 'NOTsema…

In [13]:
jsnf = jsnWD.value
print (f"{jsnf=} ")

jsnfile = osp.join (JSONPath,jsnf)
xSummaries = pd.read_json(jsnfile)
# xSummaries = xSummaries.transpose() # pivot the matrix
xSummaries

jsnf='semanticSimArticlesTestingME.json' 


,source,summary,hf1,hf2,hf3,hf4,hf5,hfAvg
0,mixed_summary1,It's a race for the governor's mansion in 11 s...,0.542,0.455,0.445,0.440,0.432,0.443000
1,mixed_summary2,It turns out Facebook is only guilty of about ...,0.482,0.449,0.440,0.430,0.427,0.436500
2,mixed_summary3,Not a big fan of Southern California? Neither ...,0.419,0.400,0.374,0.365,0.362,0.384000
3,mixed_summary4,Why did Microsoft buy Nokia's phone business? ...,0.647,0.629,0.628,0.616,0.606,0.625200
4,mixed_summary5,The Supreme Court is facing a docket of high-p...,0.432,0.413,0.380,0.378,0.372,0.395000
...,...,...,...,...,...,...,...,...
5617,mixed_summary5618,The traditional end-of-summit group photo at t...,0.530,0.448,0.446,0.423,0.420,0.434250
5618,mixed_summary5619,Sofia Coppola scored a historic victory at the...,0.300,0.293,0.256,0.251,0.248,0.269600
5619,mixed_summary5620,The duck boat sinking that killed 17 on a Miss...,0.359,0.340,0.334,0.324,0.280,0.339250
5620,mixed_summary5621,Note to tweeting politicians: Watch what you p...,0.529,0.467,0.467,0.463,0.455,0.465667


In [14]:
xSentences = preProcessing(xSummaries)


Cell models @ 2023-02-03 19:44:58


In [15]:
xSentences[0]

'race governor mansion state today gop could end night helm two third state gop currently control country top state office expected keep three republican one grab utah north dakota indiana wrest north carolina dems brings toll potential take three report npr race montana new hampshire washington still close call three democrat incumbent seeking reelection result could big impact health care since supreme court ruling grant state ability opt obamacare medicaid expansion romney victory would dramatically empower republican governor said one analyst click npr state state breakdown could happen'

### Encode the sentences:

In [16]:
xSentence_embeddings = model.encode(xSentences)
# xSentence_embeddings.shape

In [17]:
cSentence_embeddings = model.encode(cSentences)
# cSentence_embeddings.shape

In [18]:
print (f"cSentence_embeddings: {cSentence_embeddings[0]}", end="\n\n")
print (f"xSentence_embeddings: {xSentence_embeddings[0]}")

cSentence_embeddings: [-4.83780950e-02 -2.75095012e-02  1.60715561e-02  1.57002322e-02
  1.07021168e-01 -6.50019804e-03 -5.05124331e-02 -3.57177258e-02
  3.41106467e-02  1.23618329e-02  1.31691173e-02  7.79866502e-02
  1.63981132e-03 -2.89458707e-02 -9.97683480e-02 -1.36307552e-02
 -4.19064239e-02 -1.08491322e-02 -5.21043949e-02  1.15556374e-01
 -5.46976784e-03  3.24263088e-02  7.18414336e-02 -1.96892284e-02
 -4.54587769e-03  2.31480282e-02 -6.77628741e-02 -4.29963991e-02
 -1.11857548e-01 -3.34735513e-02  6.20625019e-02  6.09268621e-02
  3.69292907e-02  4.23717164e-02  1.08853273e-01 -2.45025735e-02
  3.42690572e-02  2.07658634e-02  1.02707215e-01 -1.64925754e-02
 -3.63139957e-02 -1.46217436e-01 -6.89819129e-03  2.93060448e-02
 -3.81086767e-02  6.14364371e-02 -2.63481401e-02  5.28832376e-02
 -1.92858465e-02 -6.94705471e-02 -5.63247390e-02  5.88743836e-02
  1.37572223e-02  5.57106510e-02 -4.10957448e-02 -4.67587709e-02
 -1.60674006e-02  2.61155833e-02 -2.62322207e-03  6.30843565e-02
  3

In [19]:
def getHiFive(cs):
    similarities = []
    ord         = 0
    for elt in cs:
        ord += 1
        similarities.append([ord, elt])
    s = sorted(similarities,key=lambda x: x[1] )
    s = s[len(s)-6:len(s)-1]
    print (f"S: {s}")
    return s

In [20]:
scores = []
for i in range (0, len(xSentence_embeddings)):
    cs = cosine_similarity(
        [xSentence_embeddings[i]],
        cSentence_embeddings[:]
    )
    s = getHiFive(cs[0])
    scores.append(s)
complete(state="xSentence_embeddings")

S: [[40, 0.30639046], [2818, 0.317035], [1999, 0.32089758], [1880, 0.35398796], [2086, 0.36790884]]
S: [[1030, 0.40700114], [1798, 0.41263965], [2144, 0.4373359], [2116, 0.4453658], [1274, 0.4464615]]
S: [[980, 0.28615156], [3059, 0.29809463], [2814, 0.30577487], [1829, 0.30697954], [218, 0.3374198]]
S: [[2883, 0.5440242], [2919, 0.54762864], [107, 0.55251354], [2436, 0.55618095], [1320, 0.56448483]]
S: [[1971, 0.340958], [1470, 0.34361714], [1025, 0.3470161], [2286, 0.35023463], [13, 0.3570344]]
S: [[1431, 0.29121393], [2893, 0.29886204], [3334, 0.32503584], [1247, 0.33299315], [2718, 0.34235802]]
S: [[2161, 0.53529036], [3205, 0.5442472], [1996, 0.54833263], [579, 0.5505464], [1838, 0.5744723]]
S: [[1, 0.33333346], [3009, 0.3470805], [2561, 0.34821707], [2708, 0.3493281], [2718, 0.35255513]]
S: [[222, 0.38419455], [1354, 0.3912965], [3431, 0.4112901], [447, 0.4113909], [2200, 0.4194258]]
S: [[56, 0.4010353], [1463, 0.40745008], [3116, 0.40865147], [2317, 0.44054765], [12, 0.44742435]

S: [[1702, 0.3612071], [1848, 0.36398596], [864, 0.36826056], [179, 0.3683274], [1952, 0.3718769]]
S: [[1673, 0.32006347], [1705, 0.3431466], [2515, 0.36699432], [663, 0.36834854], [2873, 0.40104026]]
S: [[1786, 0.3806677], [1146, 0.3874683], [1068, 0.39371347], [820, 0.40978622], [244, 0.44833267]]
S: [[1449, 0.40591833], [1514, 0.4148171], [3023, 0.41879433], [648, 0.42231026], [651, 0.42483845]]
S: [[3149, 0.35164452], [1547, 0.36327678], [864, 0.36561054], [552, 0.37117052], [2152, 0.39920884]]
S: [[8, 0.45741302], [3282, 0.4684244], [1898, 0.4796448], [1060, 0.4865636], [1675, 0.5129546]]
S: [[1798, 0.497789], [2873, 0.51544374], [2200, 0.5221521], [1184, 0.52400553], [883, 0.5285386]]
S: [[1871, 0.33499455], [3532, 0.3428772], [1155, 0.35053822], [383, 0.3546567], [1762, 0.3564718]]
S: [[1025, 0.24888232], [3499, 0.24897304], [2478, 0.2614474], [3082, 0.26144966], [3021, 0.26416355]]
S: [[66, 0.3385123], [1523, 0.34094727], [1, 0.3486755], [3372, 0.34917936], [3003, 0.36628154]]


S: [[1184, 0.3045655], [770, 0.32294074], [1354, 0.3303643], [859, 0.34777024], [3372, 0.35105413]]
S: [[442, 0.3463509], [40, 0.3576669], [1113, 0.36946803], [16, 0.37079343], [1217, 0.38304323]]
S: [[2309, 0.249742], [2029, 0.2625281], [3222, 0.26329523], [454, 0.2816006], [3082, 0.3035373]]
S: [[2313, 0.2799395], [3082, 0.29253706], [368, 0.29462078], [3358, 0.2999771], [3372, 0.33357328]]
S: [[189, 0.46580255], [1270, 0.46681076], [1999, 0.4700936], [545, 0.472623], [3219, 0.52442]]
S: [[40, 0.3442952], [3430, 0.35356236], [212, 0.35371947], [545, 0.36020005], [2974, 0.36829594]]
S: [[3123, 0.35484737], [1, 0.3795808], [2521, 0.38171846], [1523, 0.3912614], [1113, 0.4155498]]
S: [[3358, 0.31157166], [2848, 0.32803994], [3532, 0.3325677], [859, 0.3346608], [3308, 0.33642212]]
S: [[328, 0.35236755], [215, 0.35890228], [1937, 0.36866742], [2379, 0.4144371], [3101, 0.43235007]]
S: [[120, 0.36575565], [1533, 0.38055098], [2111, 0.39880353], [3052, 0.40478534], [3210, 0.42610842]]
S: [[1

S: [[2198, 0.25022072], [3372, 0.25814188], [623, 0.26354593], [2427, 0.26416448], [3358, 0.30601618]]
S: [[1868, 0.25813973], [2427, 0.26140797], [3195, 0.27048317], [3532, 0.28040075], [1311, 0.30920795]]
S: [[3042, 0.3026845], [750, 0.3043477], [1533, 0.3125664], [2607, 0.3171299], [2316, 0.32888496]]
S: [[3225, 0.45095637], [3385, 0.48064938], [2423, 0.4825217], [1203, 0.52332634], [3337, 0.5236866]]
S: [[3408, 0.4216919], [3121, 0.4253894], [1459, 0.4286884], [2050, 0.44400758], [987, 0.4587562]]
S: [[2038, 0.32521015], [2427, 0.33849615], [859, 0.34809098], [1275, 0.3550002], [38, 0.36742014]]
S: [[137, 0.24527417], [3125, 0.24921241], [1769, 0.25190237], [2793, 0.25480902], [154, 0.26140392]]
S: [[2642, 0.32582343], [2469, 0.33085775], [1340, 0.33578306], [2200, 0.3420196], [1748, 0.3594262]]
S: [[1939, 0.34296954], [1871, 0.34303492], [2351, 0.35173827], [921, 0.3539548], [3081, 0.35789812]]
S: [[2478, 0.40057114], [3116, 0.4266112], [1761, 0.43651015], [1951, 0.46625325], [12,

S: [[837, 0.36434966], [1838, 0.38441044], [2868, 0.39109147], [775, 0.43272173], [968, 0.4788962]]
S: [[454, 0.29995066], [368, 0.3009091], [2313, 0.30537823], [663, 0.31356558], [3082, 0.32223663]]
S: [[1557, 0.3180846], [1607, 0.3208145], [3189, 0.32314909], [3322, 0.32569942], [1085, 0.32804152]]
S: [[2663, 0.28211802], [2868, 0.28744254], [2985, 0.2975289], [1871, 0.30768132], [70, 0.3183546]]
S: [[504, 0.39831734], [3382, 0.39936334], [1084, 0.40612578], [3423, 0.40800583], [2968, 0.41764826]]
S: [[1472, 0.39256787], [1781, 0.3932972], [247, 0.4017216], [1186, 0.40502584], [1561, 0.4233324]]
S: [[431, 0.38892677], [1468, 0.39876628], [3326, 0.45152265], [328, 0.46946177], [3101, 0.5447768]]
S: [[2329, 0.32365936], [2127, 0.32735813], [3372, 0.33850783], [3190, 0.3402164], [3082, 0.34601638]]
S: [[623, 0.4019524], [2200, 0.40517664], [2478, 0.40638402], [647, 0.4128835], [2893, 0.4263327]]
S: [[3455, 0.3263373], [66, 0.331515], [1971, 0.333102], [2615, 0.34124103], [41, 0.34528875

...

S: [[2651, 0.31862783], [609, 0.31913164], [2147, 0.33010727], [3339, 0.3418379], [1540, 0.3486107]]
S: [[3358, 0.2690037], [3082, 0.27859157], [22, 0.28216162], [1629, 0.28513166], [984, 0.301274]]
S: [[2718, 0.32852122], [220, 0.32863516], [1202, 0.334103], [3423, 0.3492468], [2152, 0.35012734]]
S: [[1354, 0.3448512], [1431, 0.35487932], [179, 0.35603952], [1952, 0.35691062], [2718, 0.41706744]]
S: [[2943, 0.29588425], [1951, 0.31203645], [49, 0.32277858], [2478, 0.3384233], [282, 0.37742418]]
S: [[2250, 0.36107695], [1463, 0.36263818], [2200, 0.3660214], [12, 0.3772741], [3372, 0.3796575]]
S: [[3116, 0.3741714], [12, 0.38415754], [1463, 0.38780057], [1761, 0.39168608], [1850, 0.40217817]]
S: [[647, 0.31886798], [282, 0.31968203], [2317, 0.32773462], [2203, 0.32794774], [66, 0.32948288]]
S: [[189, 0.28570443], [1782, 0.29622135], [2974, 0.30790973], [905, 0.3140345], [146, 0.3171593]]
S: [[3259, 0.30378062], [1500, 0.31324065], [648, 0.32421884], [1950, 0.3494138], [1371, 0.36017656]

S: [[3121, 0.37765962], [585, 0.38500524], [754, 0.38749534], [2546, 0.4073192], [940, 0.42035317]]
S: [[2317, 0.3344615], [2727, 0.33683452], [66, 0.34495276], [1463, 0.35201806], [2261, 0.35429105]]
S: [[3358, 0.28658888], [3082, 0.28765592], [1921, 0.29827595], [2203, 0.30414212], [3308, 0.30769983]]
S: [[3111, 0.22927189], [154, 0.23146623], [383, 0.23488146], [175, 0.23918834], [3532, 0.24324848]]
S: [[3532, 0.28437668], [3372, 0.3032773], [859, 0.31038535], [3358, 0.33443987], [454, 0.37662047]]
S: [[3258, 0.26800686], [1850, 0.27738637], [2364, 0.29316083], [66, 0.29652274], [2536, 0.30628783]]
S: [[2419, 0.4240304], [1789, 0.42534643], [2996, 0.44465595], [3465, 0.4494473], [883, 0.45089293]]
S: [[1263, 0.37995106], [1113, 0.38047406], [1950, 0.38791254], [1586, 0.39425793], [1681, 0.39793962]]
S: [[2329, 0.31462026], [1406, 0.3176478], [2589, 0.31835377], [1678, 0.32001048], [871, 0.3335826]]
S: [[415, 0.51077753], [3476, 0.5197666], [497, 0.52061784], [3323, 0.5386723], [2965

S: [[10, 0.2895105], [3478, 0.30350077], [3111, 0.31024835], [3532, 0.3105545], [1642, 0.35919702]]
S: [[2515, 0.35985172], [794, 0.36066625], [114, 0.36704358], [3287, 0.37423092], [2651, 0.38640213]]
S: [[40, 0.39343324], [1113, 0.4029215], [1217, 0.40410993], [1022, 0.4085682], [1270, 0.420309]]
S: [[3076, 0.24999967], [1642, 0.26354182], [3409, 0.27546045], [383, 0.28455073], [2064, 0.29259655]]
S: [[526, 0.43161952], [212, 0.4467698], [179, 0.4482585], [1952, 0.45573825], [222, 0.4647591]]
S: [[2113, 0.40862387], [3215, 0.41277513], [2972, 0.4132796], [504, 0.4157406], [1573, 0.41712666]]
S: [[3195, 0.32298964], [161, 0.32310173], [89, 0.32489273], [13, 0.33431584], [2963, 0.34606647]]
S: [[220, 0.35313508], [2476, 0.3620308], [775, 0.3650387], [2320, 0.37287247], [1804, 0.38634285]]
S: [[2868, 0.3065965], [1979, 0.31365934], [1561, 0.31374168], [2735, 0.33059925], [837, 0.33898193]]
S: [[2317, 0.37707233], [12, 0.38244146], [3286, 0.39941424], [2708, 0.41679555], [2718, 0.4338993

S: [[1851, 0.54481703], [3536, 0.54486626], [379, 0.5530833], [2418, 0.56098473], [277, 0.5848561]]
S: [[1184, 0.35833433], [2198, 0.371044], [452, 0.38330132], [2152, 0.39553612], [154, 0.4010653]]
S: [[316, 0.36676645], [579, 0.37118244], [3491, 0.38248062], [3256, 0.39792976], [1561, 0.46080452]]
S: [[420, 0.4184523], [546, 0.4184523], [3249, 0.4184523], [1312, 0.44276738], [2965, 0.4667987]]
S: [[3334, 0.30535805], [3358, 0.30825898], [2200, 0.30982056], [1921, 0.3106279], [154, 0.31480584]]
S: [[1678, 0.43183208], [871, 0.43337327], [2589, 0.43813157], [1364, 0.44243884], [500, 0.44379988]]
S: [[367, 0.31874308], [2370, 0.326782], [2144, 0.33703387], [3366, 0.3403181], [2642, 0.34419885]]
S: [[794, 0.3773026], [3372, 0.38133326], [13, 0.38147804], [3385, 0.38398355], [1971, 0.3876294]]
S: [[1921, 0.32160634], [2963, 0.3612889], [1582, 0.36500126], [2329, 0.37224245], [3308, 0.37488997]]
S: [[1184, 0.35568452], [2094, 0.36558193], [1573, 0.36670974], [2499, 0.3668021], [222, 0.3871

S: [[623, 0.39784658], [647, 0.410248], [1626, 0.41739383], [680, 0.4525885], [1030, 0.4592768]]
S: [[2469, 0.32650906], [3366, 0.32664388], [3149, 0.34760597], [764, 0.34763592], [907, 0.41087663]]
S: [[2815, 0.38962108], [2217, 0.3902987], [637, 0.39199686], [687, 0.39659834], [1034, 0.40176737]]
S: [[222, 0.36048484], [284, 0.3680372], [674, 0.3680372], [526, 0.3691401], [367, 0.37320292]]
S: [[2200, 0.29806858], [2943, 0.32358983], [1354, 0.33529258], [2642, 0.34283674], [282, 0.3559603]]
S: [[1275, 0.5880033], [1165, 0.5969364], [2799, 0.6193817], [3389, 0.6282015], [1347, 0.6324651]]
S: [[3102, 0.3283345], [1168, 0.35104096], [2944, 0.354312], [1971, 0.35472026], [3372, 0.3567376]]
S: [[856, 0.3259725], [3378, 0.32883075], [889, 0.35242593], [726, 0.3538711], [2711, 0.39514393]]
S: [[2979, 0.3060998], [681, 0.3170475], [2893, 0.3304158], [623, 0.33739248], [1396, 0.34789228]]
S: [[3155, 0.3274181], [921, 0.34850922], [1850, 0.3541757], [2765, 0.36644906], [680, 0.39691496]]
S: [[

In [22]:
# Reverse the list
revScore = []
print (f"Total Scores : {len(scores)}")
for score in scores:
    score = score[::-1]
    print(f"Score:{score}")
    revScore.append(score)

Total Scores : 5622
Score:[[2086, 0.36790884], [1880, 0.35398796], [1999, 0.32089758], [2818, 0.317035], [40, 0.30639046]]
Score:[[1274, 0.4464615], [2116, 0.4453658], [2144, 0.4373359], [1798, 0.41263965], [1030, 0.40700114]]
Score:[[218, 0.3374198], [1829, 0.30697954], [2814, 0.30577487], [3059, 0.29809463], [980, 0.28615156]]
Score:[[1320, 0.56448483], [2436, 0.55618095], [107, 0.55251354], [2919, 0.54762864], [2883, 0.5440242]]
Score:[[13, 0.3570344], [2286, 0.35023463], [1025, 0.3470161], [1470, 0.34361714], [1971, 0.340958]]
Score:[[2718, 0.34235802], [1247, 0.33299315], [3334, 0.32503584], [2893, 0.29886204], [1431, 0.29121393]]
Score:[[1838, 0.5744723], [579, 0.5505464], [1996, 0.54833263], [3205, 0.5442472], [2161, 0.53529036]]
Score:[[2718, 0.35255513], [2708, 0.3493281], [2561, 0.34821707], [3009, 0.3470805], [1, 0.33333346]]
Score:[[2200, 0.4194258], [447, 0.4113909], [3431, 0.4112901], [1354, 0.3912965], [222, 0.38419455]]
Score:[[12, 0.44742435], [2317, 0.44054765], [3116

Score:[[349, 0.42287013], [552, 0.4142168], [2476, 0.41109222], [3491, 0.39998367], [837, 0.39437425]]
Score:[[1515, 0.30504555], [2527, 0.27174336], [1261, 0.26662427], [2925, 0.2618823], [3161, 0.26089793]]
Score:[[2484, 0.25787696], [452, 0.25420108], [1096, 0.25416788], [875, 0.25416788], [2156, 0.24774937]]
Score:[[3431, 0.3458611], [1354, 0.3434834], [1523, 0.33123145], [3116, 0.3148846], [2893, 0.31160113]]
Score:[[367, 0.40740383], [282, 0.39638743], [3372, 0.37011153], [49, 0.3634599], [3116, 0.3558978]]
Score:[[2812, 0.32284027], [3190, 0.32203653], [1254, 0.31182405], [2272, 0.304149], [2437, 0.30395555]]
Score:[[1951, 0.30310357], [367, 0.2834099], [1162, 0.28263003], [727, 0.26840293], [138, 0.26482135]]
Score:[[3205, 0.4340167], [1138, 0.3831308], [146, 0.3727619], [250, 0.35716915], [1533, 0.32852155]]
Score:[[2979, 0.28951395], [1096, 0.28269875], [875, 0.28269875], [2203, 0.27496848], [647, 0.2723021]]
Score:[[2320, 0.37721798], [1804, 0.36458272], [3256, 0.3484363], [

Score:[[1812, 0.37168768], [3210, 0.30787826], [859, 0.28154725], [584, 0.27943993], [1761, 0.27850145]]
Score:[[1463, 0.38812262], [859, 0.38742083], [2261, 0.3745516], [1202, 0.3629337], [2250, 0.36088902]]
Score:[[3209, 0.4251083], [1735, 0.42227384], [3226, 0.41078073], [2894, 0.4084451], [1676, 0.4068588]]
Score:[[16, 0.37445417], [41, 0.36962676], [3190, 0.36677268], [1659, 0.36443323], [632, 0.36235112]]
Score:[[2963, 0.32372308], [1340, 0.29118434], [3390, 0.29068938], [3358, 0.28689313], [618, 0.26116127]]
Score:[[1523, 0.32635546], [1396, 0.31592628], [3162, 0.3083413], [2873, 0.3050116], [2200, 0.30442625]]
Score:[[2194, 0.357984], [1067, 0.34825706], [13, 0.33640704], [2963, 0.33186066], [3390, 0.3296642]]
Score:[[623, 0.33200747], [282, 0.32916147], [1169, 0.32139504], [3116, 0.31602848], [2893, 0.3050739]]
Score:[[2864, 0.35704178], [3431, 0.35230154], [56, 0.34584254], [1761, 0.31883308], [1909, 0.31303865]]
Score:[[1025, 0.36370715], [3337, 0.36223537], [747, 0.3491593]

Score:[[1150, 0.36017507], [2252, 0.35092926], [40, 0.34466186], [189, 0.3176244], [2061, 0.3022925]]
Score:[[3009, 0.36650637], [413, 0.36098957], [3059, 0.3592533], [1631, 0.3570646], [1275, 0.35228297]]
Score:[[2252, 0.42656887], [1270, 0.38968968], [1595, 0.377281], [40, 0.37435043], [2933, 0.36574757]]
Score:[[921, 0.39345008], [1025, 0.37788823], [2423, 0.34964195], [1203, 0.33719492], [3385, 0.3311643]]
Score:[[1561, 0.42025048], [3256, 0.38954353], [2161, 0.38314384], [2234, 0.37169722], [1996, 0.35133702]]
Score:[[367, 0.38806963], [1030, 0.3810085], [859, 0.37446916], [1354, 0.37209103], [1626, 0.36693493]]
Score:[[3337, 0.5307724], [1971, 0.52478254], [3225, 0.51150024], [1203, 0.51005834], [2423, 0.50938714]]
Score:[[1, 0.42389536], [2718, 0.39693278], [2286, 0.38695288], [3075, 0.3866886], [2615, 0.37650165]]
Score:[[189, 0.41157663], [367, 0.40969375], [1150, 0.40607268], [212, 0.40308994], [40, 0.39995572]]
Score:[[681, 0.3380173], [2457, 0.32613993], [3082, 0.3026], [41

Score:[[2926, 0.41841704], [2708, 0.41814914], [3009, 0.41789323], [859, 0.4171304], [1463, 0.3881118]]
Score:[[734, 0.34916094], [3358, 0.3130414], [2140, 0.3026167], [186, 0.29485407], [3259, 0.29267842]]
Score:[[859, 0.3899448], [1463, 0.37621507], [2317, 0.37520254], [56, 0.3680418], [1275, 0.36262625]]
Score:[[189, 0.39714873], [2252, 0.3927299], [931, 0.3804896], [1270, 0.3636194], [1523, 0.35935372]]
Score:[[2261, 0.41482735], [1202, 0.4105358], [3009, 0.3721741], [1463, 0.37100458], [462, 0.37052846]]
Score:[[2963, 0.41089302], [2261, 0.36728334], [2317, 0.3574777], [1354, 0.3363813], [1463, 0.33473092]]
Score:[[2642, 0.31495395], [337, 0.30615455], [1507, 0.30364466], [222, 0.29783258], [727, 0.29733926]]
Score:[[248, 0.53371096], [885, 0.5213902], [2725, 0.5139958], [793, 0.48910648], [1322, 0.46841294]]
Score:[[1113, 0.47758764], [3123, 0.44855678], [2113, 0.4258408], [212, 0.42260602], [1, 0.41938114]]
Score:[[2471, 0.43330982], [2383, 0.41840762], [2038, 0.41629937], [2596

Score:[[1609, 0.41058725], [3333, 0.40695265], [1655, 0.40030986], [2632, 0.40006274], [788, 0.39647275]]
Score:[[200, 0.38543612], [501, 0.37763566], [1, 0.28854734], [1229, 0.2782418], [1761, 0.27687043]]
Score:[[282, 0.36274445], [612, 0.3450507], [1275, 0.3419176], [1921, 0.33426693], [859, 0.33165354]]
Score:[[1937, 0.2907943], [2646, 0.2706543], [56, 0.27048707], [2943, 0.27007505], [1444, 0.26362845]]
Score:[[3358, 0.321946], [3082, 0.3105592], [282, 0.29823723], [3455, 0.29584175], [368, 0.28831863]]
Score:[[681, 0.39123538], [3116, 0.37821007], [2200, 0.36897], [2478, 0.35910574], [110, 0.35654446]]
Score:[[1168, 0.3889775], [3455, 0.35913485], [1030, 0.35767952], [2478, 0.3567509], [2317, 0.3533721]]
Score:[[2245, 0.6077845], [2188, 0.590464], [589, 0.5275734], [2722, 0.5046402], [2780, 0.500706]]
Score:[[3059, 0.60825515], [1339, 0.5784255], [1304, 0.4832244], [2147, 0.48120558], [366, 0.46505132]]
Score:[[2011, 0.45808706], [1253, 0.4575801], [2079, 0.45386732], [698, 0.450

Score:[[2515, 0.44336677], [2811, 0.42840254], [3311, 0.4146451], [2086, 0.403927], [1937, 0.38448322]]
Score:[[501, 0.45645168], [1406, 0.4515291], [500, 0.4422832], [2981, 0.4322844], [1678, 0.4164995]]
Score:[[2633, 0.26724118], [2978, 0.25928605], [3190, 0.2509392], [2437, 0.23930915], [1143, 0.23273592]]
Score:[[1523, 0.39949337], [3076, 0.39495388], [16, 0.391616], [2974, 0.38963526], [513, 0.38871452]]
Score:[[1289, 0.62143505], [1770, 0.57908726], [153, 0.56236213], [921, 0.5553299], [443, 0.5489133]]
Score:[[10, 0.27100107], [98, 0.26509753], [3375, 0.26472425], [3081, 0.2634042], [3114, 0.2449551]]
Score:[[189, 0.35726675], [2252, 0.35075092], [1270, 0.34472698], [1034, 0.33759722], [2818, 0.33442962]]
Score:[[3337, 0.38488367], [1025, 0.38317442], [794, 0.38218606], [3219, 0.36165237], [17, 0.3579169]]
Score:[[3190, 0.29552412], [648, 0.29248047], [1786, 0.29118747], [2769, 0.2668289], [1449, 0.26044226]]
Score:[[1850, 0.4045145], [3116, 0.3910765], [1030, 0.35638562], [282,

Score:[[2113, 0.40628886], [1523, 0.3965979], [1582, 0.37686023], [1270, 0.3736663], [2702, 0.37276292]]
Score:[[1838, 0.43991482], [2161, 0.3983414], [1996, 0.39722297], [477, 0.37984487], [2868, 0.359133]]
Score:[[1838, 0.4466102], [1516, 0.4417948], [3479, 0.4085272], [579, 0.40578753], [1257, 0.40373695]]
Score:[[234, 0.5919454], [2050, 0.5816009], [3121, 0.5739274], [987, 0.567744], [2643, 0.561711]]
Score:[[3029, 0.32561922], [2797, 0.29582173], [1523, 0.28576875], [3532, 0.28503338], [1185, 0.2832212]]
Score:[[2252, 0.37212604], [3272, 0.3705392], [2113, 0.36895552], [905, 0.3598987], [1217, 0.3539148]]
Score:[[837, 0.43734598], [826, 0.42860675], [2476, 0.41212392], [3175, 0.40884686], [968, 0.40520766]]
Score:[[2532, 0.31754506], [1786, 0.31708485], [547, 0.29336408], [2152, 0.29274732], [646, 0.29008478]]
Score:[[328, 0.39452186], [2379, 0.37619102], [371, 0.37618944], [802, 0.36016297], [215, 0.35451964]]
Score:[[1523, 0.37458503], [3162, 0.36974323], [447, 0.3688432], [2564

In [23]:
print(scores[0])
print(revScore[0])

[[40, 0.30639046], [2818, 0.317035], [1999, 0.32089758], [1880, 0.35398796], [2086, 0.36790884]]
[[2086, 0.36790884], [1880, 0.35398796], [1999, 0.32089758], [2818, 0.317035], [40, 0.30639046]]


### Test Results
From the tests above, it seems clear that the estimation method works. One question is where we do the cutoffs?

Results in the 0.0 to 0.65x range seem to be below cutoff.

Results in the 0.7 and above range seem to be right for inclusion,

But what about something in the range 0.6 to 0.699?

### Edited on 01/24/2023
Switched transformer model from 'bert-base-nli-mean-tokens' to the model used to obtain HF scores: 'all-MiniLM-L6-v2'

New scores are much lower than seen previously, more similar to HF score range

In [24]:
# write out the KeyBERT Scores and add to candidate score df 

In [25]:
xSummaries.head()

,source,summary,hf1,hf2,hf3,hf4,hf5,hfAvg
0,mixed_summary1,race governor mansion state today gop could en...,0.542,0.455,0.445,0.440,0.432,0.4430
1,mixed_summary2,turn facebook guilty half accused gay kiss inc...,0.482,0.449,0.440,0.430,0.427,0.4365
2,mixed_summary3,big fan southern california neither northern c...,0.419,0.400,0.374,0.365,0.362,0.3840
3,mixed_summary4,microsoft buy nokia phone business know micros...,0.647,0.629,0.628,0.616,0.606,0.6252
4,mixed_summary5,supreme court facing docket high profile polit...,0.432,0.413,0.380,0.378,0.372,0.3950


In [26]:
print(revScore[0])

revScore[0][1][1]

[[2086, 0.36790884], [1880, 0.35398796], [1999, 0.32089758], [2818, 0.317035], [40, 0.30639046]]


0.35398796

In [27]:
# create df out of KeyBERT score list
cols = list('abcde')
dftest = pd.DataFrame(revScore, columns=cols)
dftest

,a,b,c,d,e
0,"[2086, 0.36790884]","[1880, 0.35398796]","[1999, 0.32089758]","[2818, 0.317035]","[40, 0.30639046]"
1,"[1274, 0.4464615]","[2116, 0.4453658]","[2144, 0.4373359]","[1798, 0.41263965]","[1030, 0.40700114]"
2,"[218, 0.3374198]","[1829, 0.30697954]","[2814, 0.30577487]","[3059, 0.29809463]","[980, 0.28615156]"
3,"[1320, 0.56448483]","[2436, 0.55618095]","[107, 0.55251354]","[2919, 0.54762864]","[2883, 0.5440242]"
4,"[13, 0.3570344]","[2286, 0.35023463]","[1025, 0.3470161]","[1470, 0.34361714]","[1971, 0.340958]"
...,...,...,...,...,...
5617,"[916, 0.4174016]","[3338, 0.40196198]","[3457, 0.39256424]","[1338, 0.38498756]","[3133, 0.3764665]"
5618,"[2200, 0.28079474]","[1737, 0.26126975]","[1362, 0.2564607]","[812, 0.25234473]","[3532, 0.2512151]"
5619,"[3116, 0.3500363]","[1354, 0.3493632]","[49, 0.34219953]","[3334, 0.33915454]","[222, 0.3363694]"
5620,"[1270, 0.45428866]","[2172, 0.4474579]","[2113, 0.44494802]","[212, 0.44434208]","[1671, 0.4370889]"


In [28]:
# demonstrate can flatten column and isolate score
dftest1 = pd.DataFrame(dftest['a'].values.tolist(), columns=['a','b'])
dftest1 = dftest1.drop(columns=['a'])
dftest1

,b
0,0.367909
1,0.446461
2,0.337420
3,0.564485
4,0.357034
...,...
5617,0.417402
5618,0.280795
5619,0.350036
5620,0.454289


In [29]:
#iterate through scores df, expand columns, then add KB score column to xSummaries df
scoreIT = 1

for column in dftest:
    col_name = 'kb'+ str(scoreIT)
    
    tempdf = pd.DataFrame(dftest[column].values.tolist(), columns=['a','b'])
    xSummaries[col_name] = tempdf['b']
    
#     print(col_name)
    scoreIT += 1

In [30]:
# preview modified candidate df with hf and kb scores
pd.set_option("max_colwidth", 30)
pd.set_option("expand_frame_repr", True)
xSummaries.head()

,source,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5
0,mixed_summary1,race governor mansion stat...,0.542,0.455,0.445,0.440,0.432,0.4430,0.367909,0.353988,0.320898,0.317035,0.306390
1,mixed_summary2,turn facebook guilty half ...,0.482,0.449,0.440,0.430,0.427,0.4365,0.446461,0.445366,0.437336,0.412640,0.407001
2,mixed_summary3,big fan southern californi...,0.419,0.400,0.374,0.365,0.362,0.3840,0.337420,0.306980,0.305775,0.298095,0.286152
3,mixed_summary4,microsoft buy nokia phone ...,0.647,0.629,0.628,0.616,0.606,0.6252,0.564485,0.556181,0.552514,0.547629,0.544024
4,mixed_summary5,supreme court facing docke...,0.432,0.413,0.380,0.378,0.372,0.3950,0.357034,0.350235,0.347016,0.343617,0.340958


### Calculate weighted average of KB scores:
Use same method as used for HF score average, remove outliers first using IQR, then find the mean

In [31]:
def removeOutliers(test_list):
    q1 = np.percentile(test_list, 25)
    q3 = np.percentile(test_list, 75)
    iqr = (q3 - q1) * 1.5
    q_set = (q1 - iqr, q3 + iqr)

    result_list =[]

    for num in test_list:
        if num >= q_set[0] and num <= q_set[1]:
            result_list.append(num)

    return result_list

In [32]:
def calculateScoreAvg(df):
    score_avg = []
    
    for i in range(len(df)):
        row_list = df.loc[i, ['kb1', 'kb2', 'kb3', 'kb4', 'kb5']].values.flatten().tolist()
        test_list = list(map(float, row_list))
        
        new_list = removeOutliers(test_list)
        weighted_score = np.mean(new_list)
#         print(f"Weighted score of row {i}: {weighted_score}")
        
        score_avg.append(weighted_score)
        i +=1
        
    return score_avg

In [33]:
kb_avg = calculateScoreAvg(xSummaries)

xSummaries['kbAvg'] = kb_avg
xSummaries

,source,summary,hf1,hf2,hf3,hf4,hf5,hfAvg,kb1,kb2,kb3,kb4,kb5,kbAvg
0,mixed_summary1,race governor mansion stat...,0.542,0.455,0.445,0.440,0.432,0.443000,0.367909,0.353988,0.320898,0.317035,0.306390,0.333244
1,mixed_summary2,turn facebook guilty half ...,0.482,0.449,0.440,0.430,0.427,0.436500,0.446461,0.445366,0.437336,0.412640,0.407001,0.429761
2,mixed_summary3,big fan southern californi...,0.419,0.400,0.374,0.365,0.362,0.384000,0.337420,0.306980,0.305775,0.298095,0.286152,0.299250
3,mixed_summary4,microsoft buy nokia phone ...,0.647,0.629,0.628,0.616,0.606,0.625200,0.564485,0.556181,0.552514,0.547629,0.544024,0.552966
4,mixed_summary5,supreme court facing docke...,0.432,0.413,0.380,0.378,0.372,0.395000,0.357034,0.350235,0.347016,0.343617,0.340958,0.347772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5617,mixed_summary5618,traditional end summit gro...,0.530,0.448,0.446,0.423,0.420,0.434250,0.417402,0.401962,0.392564,0.384988,0.376467,0.394676
5618,mixed_summary5619,sofia coppola scored histo...,0.300,0.293,0.256,0.251,0.248,0.269600,0.280795,0.261270,0.256461,0.252345,0.251215,0.255323
5619,mixed_summary5620,duck boat sinking killed m...,0.359,0.340,0.334,0.324,0.280,0.339250,0.350036,0.349363,0.342200,0.339155,0.336369,0.343425
5620,mixed_summary5621,note tweeting politician w...,0.529,0.467,0.467,0.463,0.455,0.465667,0.454289,0.447458,0.444948,0.444342,0.437089,0.445583


In [35]:
jsonfile = join(JSONPath, "semanticSimArticlesTestingME.json")
print(jsonfile)

xSummaries.to_json(jsonfile)
complete(state="updates written: ")

/home/deleidos/Notebooks/JSONData/semanticSimArticlesTestingME.json

Cell updates written:  @ 2023-02-03 19:56:00


In [35]:
# cArticleURLs = cSummaries.url.to_list()
# # print ( f"scores[0]: {scores[0]}")
# for score in scores[0]:
#     articleID = score[0]
#     print (f"Score: {articleID} URL most similar: {cArticleURLs[articleID]}")

In [85]:
# print (scores[0][0][0])
# print (f" before: {xSummaries.iloc[0]}")
# xSummaries.iloc[0]['KB1'] = scores[0][0][0]
# DF2 = DF2 = pd.DataFrame([[11,12,13,14]], columns = ('x', 'y', 'z', 'u'), index=['A']) 
# xSummaries.iloc[0][['KB1', 'KB2', 'KB3', 'KB4'] ] = DF2[['x', 'y', 'z', 'u']]
